In [ ]:
def train_function(magic_number):
    import horovod.torch as hvd
    hvd.init()
    print('Hello, rank = %d, local_rank = %d, size = %d, local_size = %d, magic_number = %d' % (hvd.rank(), hvd.local_rank(), hvd.size(), hvd.local_size(), magic_number))
    return hvd.rank()

In [ ]:
import socket
import pyspark
from pyspark.sql import SparkSession
import horovod.spark

localIpAddress = socket.gethostbyname(socket.gethostname())

conf = pyspark.SparkConf().setAll([
    ('spark.master', 'k8s://https://kubernetes.default.svc.cluster.local:443'),
    ("spark.driver.host", localIpAddress),
    ("spark.kubernetes.namespace", "spark"),
    ("spark.kubernetes.container.image", "akirillov/spark:spark-2.4.3-hadoop-2.9-k8s-horovod")])

spark = SparkSession.builder.appName("HorovodOnSpark").config(conf=conf).getOrCreate()

In [ ]:
model = horovod.spark.run(train_function, args=(42,), verbose=2)

In [ ]:
spark.stop()